## 使用枚举类

In [3]:
from enum import Enum

Month = Enum('Month', ('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul',\
                      'Aug', 'Sep', 'Oct', 'Nov', 'Dec'))

In [13]:
for name, member in Month.__members__.items():
    print(name, '=>', member, ',', member.value)


Jan => Month.Jan , 1
Feb => Month.Feb , 2
Mar => Month.Mar , 3
Apr => Month.Apr , 4
May => Month.May , 5
Jun => Month.Jun , 6
Jul => Month.Jul , 7
Aug => Month.Aug , 8
Sep => Month.Sep , 9
Oct => Month.Oct , 10
Nov => Month.Nov , 11
Dec => Month.Dec , 12
<enum 'Month'>


如果需要更精确地控制枚举类型，可以从Enum派生出自定义类：

In [14]:
from enum import Enum, unique

@unique
class Weekday(Enum):
    Sun = 0
    Mon = 1
    Tue = 2
    Wed = 3
    Thu = 4
    Fri = 5
    Sat = 6
    


@unique装饰器可以帮助我们检查保证没有重复值。

访问这些枚举类型可以有若干种方法：

In [26]:
day1 = Weekday.Mon
print(day1)
print(Weekday.Tue)
print(Weekday['Tue'])
print(Weekday.Tue.value)
print(day1 == Weekday.Mon)
print(day1 == Weekday.Tue)
print(day1 == Weekday(1))

for name, member in Weekday.__members__.items():
    print(name, '=>', member)

Weekday.Mon
Weekday.Tue
Weekday.Tue
2
True
False
True
Sun => Weekday.Sun
Mon => Weekday.Mon
Tue => Weekday.Tue
Wed => Weekday.Wed
Thu => Weekday.Thu
Fri => Weekday.Fri
Sat => Weekday.Sat


练习

把Student的gender属性改造为枚举类型，可以避免使用字符串：

In [27]:
from enum import Enum, unique

@unique
class Gender(Enum):
    Male = 0
    Female = 1

class Student(object):
    def __init__(self, name, gender):
        self.name = name
        self.gender = gender

In [28]:
# 测试:
bart = Student('Bart', Gender.Male)
if bart.gender == Gender.Male:
    print('测试通过!')
else:
    print('测试失败!')

测试通过!


### 使用元类
type()函数既可以返回一个对象的类型，又可以创建出新的类型，比如，我们可以通过type()函数创建出Hello类，而无需通过class Hello(object)...的定义：

In [34]:
def fn(self, name='world'):
    print('Hello, %s.' % name)
# 使用type()函数创建class
Hello = type('Hello', (object,), dict(hello=fn))
h = Hello()
h.hello()
print(type(Hello))
print(type(h))

Hello, world.
<class 'type'>
<class '__main__.Hello'>


要创建一个class对象，type()函数依次传入3个参数：

1. class的名称；

2. 继承的父类集合，注意Python支持多重继承，如果只有一个父类，别忘了tuple的单元素写法；

3. class的方法名称与函数绑定，这里我们把函数fn绑定到方法名hello上。

通过type()函数创建的类和直接写class是完全一样的，因为Python解释器遇到class定义时，仅仅是扫描一下class定义的语法，然后调用type()函数创建出class。

In [35]:
# metaclass是类的模板，所以必须从`type`类型派生：
class ListMetaclass(type):
    def __new__(cls, name, bases, attrs):
        attrs['add'] = lambda self, value: self.append(value)
        return type.__new__(cls, name, bases, attrs)

class MyList(list, metaclass=ListMetaclass):
    pass

In [41]:
L = MyList()
L.add(1)
print(L)
L2 = list()
#L2.add(10)

[1]


ORM全称“Object Relational Mapping”，即对象-关系映射，就是把关系数据库的一行映射为一个对象，也就是一个类对应一个表，这样，写代码更简单，不用直接操作SQL语句。

要编写一个ORM框架，所有的类都只能动态定义，因为只有使用者才能根据表的结构定义出对应的类来。

让我们来尝试编写一个ORM框架。

编写底层模块的第一步，就是先把调用接口写出来。比如，使用者如果使用这个ORM框架，想定义一个User类来操作对应的数据库表User，我们期待他写出这样的代码：

In [14]:
class User(Model):
    __table__ = 'user'
    # 定义类的属性到列的映射
    id = IntegerField('id')
    name = StringField('name')
    email = StringField('email')
    password = StringField('password')

# 创建一个实例：
u = User(id=12345, name='Michael', email='test@orm.org',\
         password='my-pwd')
# 保存到数据库：
u.save()

Found model: User
Found mapping: id ==> <IntegerField:id>
Found mapping: name ==> <StringField:name>
Found mapping: email ==> <StringField:email>
Found mapping: password ==> <StringField:password>
SQL: insert into user (id,name,email,password) values (?,?,?,?)
ARGS: [12345, 'Michael', 'test@orm.org', 'my-pwd']


1

其中，父类Model和属性类型StringField、IntegerField是由ORM框架提供的，剩下的魔术方法比如save()全部由metaclass自动完成。虽然metaclass的编写会比较复杂，但ORM的使用者用起来却异常简单。

现在，我们就按上面的接口来实现该ORM。

首先来定义Field类，它负责保存数据库表的字段名和字段类型：

In [1]:
class Field(object):
    
    def __init__(self, name, column_type):
        self.name = name
        self.column_type = column_type
    
    def __str__(self):
        return '<%s:%s>' % (self.__class__.__name__, self.name)

在Field的基础上，进一步定义各种类型的Field，比如StringField，IntegerField等等：

In [2]:
class StringField(Field):
    
    def __init__(self, name):
        super().__init__(name, 'varchar(100)')
    
class IntegerField(Field):
    
    def __init__(self, name):
        super().__init__(name, 'bigint')

下一步，就是编写最复杂的ModelMetaclass了：

In [6]:
class ModelMetaclass(type):
    
    def __new__(cls, name, bases, attrs):
        if name == 'Model':
            return type.__new__(cls, name, bases, attrs)
        print('Found model: %s' % name)
        mappings = dict()
        tablename = attrs.pop('__table__') # 表名不属于字段属性，所以弹出
        for k, v in attrs.items():
            if isinstance(v, Field):
                print('Found mapping: %s ==> %s' % (k, v))
                mappings[k] = v
        for k in mappings.keys():
            attrs.pop(k)
        attrs['__mappings__'] = mappings # 保存属性和列的映射关系
        attrs['__table__'] = tablename # 表名
        return type.__new__(cls, name, bases, attrs)

以及基类Model：

In [13]:
class Model(dict, metaclass=ModelMetaclass):
    
    def __init__(self, **kw):
        super().__init__(**kw)
        
    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Model' object has no attribute '%s'" % key)
    
    def __setattr__(self, key, value):
        self[key] = value
        
    def save(self):
        fields = []
        params = []
        args = []
        for k, v in self.__mappings__.items():
            fields.append(v.name)
            params.append('?')
            args.append(getattr(self, k, None))
        sql = 'insert into %s (%s) values (%s)' % (self.__table__, ','.join(fields), ','.join(params))
        print('SQL: %s' % sql)
        print('ARGS: %s' % str(args))
        return self.insert1(sql, args)
    
    def insert1(self, sql, args):
        conn = pymysql.connect(host='localhost', port=3306, user='root', password='111111', db='test')
        cursor = conn.cursor()
        cursor.execute(sql.replace('?', '%s'), args)
        affected = cursor.rowcount
        conn.commit()
        cursor.close()
        conn.close()
        return affected

In [9]:
import pymysql

